In [ ]:
from pathlib import Path
repo_dir = Path('/kaggle/input/gns-solid/code')
if not repo_dir.exists():
    raise FileNotFoundError(f'{repo_dir} が見つかりません。パイプラインでデータセットを更新してください。')
%cd $repo_dir
# 依存パッケージをインストール (pyproject.toml に基づく)
!pip install -q $repo_dir


In [ ]:
import os
os.environ.pop('MPLBACKEND', None)
%cd /kaggle/input/gns-solid/code


In [ ]:
!pip install torch-geometric 

In [ ]:
from pathlib import Path
import yaml

REPO = Path('/kaggle/input/gns-solid/code')
cfg_path = REPO / 'config.yaml'

with cfg_path.open('r', encoding='utf-8') as f:
    cfg = yaml.safe_load(f)

cfg['method'] = 'hamiltonian_sph'
cfg['amp_enable'] = False
cfg['ntraining_steps'] = 100
cfg['train_dataset_count'] = 10
cfg['valid_dataset_count'] = 2
cfg['validation_interval'] = 10

# Kaggle 入力データセットのパスに合わせて上書き
dataset_root = '/kaggle/input/dam-break-left/dam_break_left'
cfg['data_path'] = dataset_root
cfg.setdefault('scenario_options', {}).setdefault('fluid', {})['dataset'] = dataset_root

with cfg_path.open('w', encoding='utf-8') as f:
    yaml.safe_dump(cfg, f, allow_unicode=True)


In [ ]:
%cd /kaggle/input/gns-solid/code
# GPUを2枚使う設定
!torchrun --nproc_per_node=2 src/train.py --config config.yaml


In [ ]:
from pathlib import Path
import yaml

REPO = Path('/kaggle/input/gns-solid/code')
cfg_path = REPO / 'config_rollout.yaml'

with cfg_path.open('r', encoding='utf-8') as f:
    cfg = yaml.safe_load(f)

cfg['method'] = 'hamiltonian_sph'
cfg['rollout_inference_max_examples'] = 1

# 推論時のデータセットパスを上書き
dataset_root = '/kaggle/input/dam-break-left/dam_break_left'
cfg.setdefault('scenario_options', {}).setdefault('fluid', {})['dataset'] = dataset_root

with cfg_path.open('w', encoding='utf-8') as f:
    yaml.safe_dump(cfg, f, allow_unicode=True)


In [ ]:
%cd /kaggle/input/gns-solid/code
!python src/train.py --config config_rollout.yaml
!python analyze_rollouts.py
!python visualize_rollout.py
